In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
text = open('./horoscopes1/horoscopes.csv', 'r', encoding="utf8").read()
text = text.split("\n")
res_text = ""
for line in text:
    if(len(line.split("|")) >= 2 and len(line) > 0):
        res_text += line.split("|")[1] + " "
text = res_text

In [3]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)


In [4]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count
print(max_id)
print(dataset_size)

56
3020120


In [5]:
[encoded] = np.array(tokenizer.texts_to_sequences([text])) - 1

In [6]:
train_size = dataset_size * 90 // 100

In [7]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [10]:
batch_size = 128
dataset = dataset.shuffle(buffer_size=batch_size * window_length).batch(batch_size).prefetch(1)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:,1:]))


In [11]:
dataset = dataset.map(
    lambda X, Y:  (tf.one_hot(X, depth=max_id), Y)) 
dataset = dataset.prefetch(1)

In [32]:
model = keras.models.Sequential([
    keras.layers.GRU(512, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.3),
    keras.layers.GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    keras.layers.GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    keras.layers.GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    keras.layers.GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    keras.layers.GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    keras.layers.GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    keras.layers.GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    keras.layers.GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    keras.layers.TimeDistributed(keras.layers.Dense(512, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
history = model.fit(dataset, epochs=10, steps_per_epoch=128)


Epoch 1/10
128/128 [==============================] - 106s 796ms/step - loss: 2.7779
Epoch 2/10
128/128 [==============================] - 101s 789ms/step - loss: 2.0620
Epoch 3/10
128/128 [==============================] - 102s 802ms/step - loss: 1.7800
Epoch 4/10
128/128 [==============================] - 99s 775ms/step - loss: 1.5942
Epoch 5/10
128/128 [==============================] - 99s 775ms/step - loss: 1.4861
Epoch 6/10
128/128 [==============================] - 101s 793ms/step - loss: 1.3808
Epoch 7/10
 73/128 [================>.............] - ETA: 44s - loss: 1.3194

In [16]:
# Plot loss from keras history object
loss = history.history['loss']

import matplotlib.pyplot as plt
import math

plt.plot(range(3), [math.log(x) for x in loss], 'bo')
plt.title('loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [17]:
def preprocess(text):
    x = np.array(tokenizer.texts_to_sequences(text)) -1
    return tf.one_hot(x,max_id)

In [18]:
X_new = preprocess(["How are you"])


In [19]:
Y_pred = model.predict(X_new)


In [20]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    Y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(Y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
def complete_text(text, n_chars=100,temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [21]:
model.predict(preprocess(["How are you"]))

array([[[1.9306834e-01, 9.5128044e-02, 7.3773041e-02, ...,
         2.8459131e-05, 6.8560212e-06, 1.4775235e-05],
        [1.9463417e-01, 9.4684444e-02, 7.4760973e-02, ...,
         1.0178546e-05, 4.1863195e-06, 5.3857780e-06],
        [1.9370756e-01, 9.6078984e-02, 7.5441375e-02, ...,
         9.0966905e-06, 4.1445769e-06, 4.8564011e-06],
        ...,
        [1.9214484e-01, 9.6883751e-02, 7.6571256e-02, ...,
         8.7617655e-06, 4.1808394e-06, 4.7089452e-06],
        [1.9212973e-01, 9.6886754e-02, 7.6575391e-02, ...,
         8.7579356e-06, 4.1816156e-06, 4.7077669e-06],
        [1.9212520e-01, 9.6885636e-02, 7.6576643e-02, ...,
         8.7552571e-06, 4.1821027e-06, 4.7068115e-06]]], dtype=float32)

In [30]:
complete_text("f",10,temperature=0.7)

'fo t et msh'